**Rust Quantum Library**

Below are dependencies, creates, and global variables.

In [2]:
:dep ndarray = { version = "0.14", features = ["serde"] }
:dep ndarray-linalg = { version = "0.13.1", features = ["intel-mkl-static"]}
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series"] }
:dep thiserror = "1.0"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep dimensioned = "0.7"
:dep num-complex = "0.4"
:dep qm = { version = "0.1", git ="https://github.com/buff-beacon-project/qm" }
:dep ndarray-csv = "0.5.1"
extern crate plotters;
extern crate ndarray;

pub use ndarray_linalg::c64;
use std::f64::consts::PI;
use plotters::prelude::*;
use ndarray::prelude::*;
use ndarray_linalg::*;
use qm::*;

extern crate csv;
extern crate ndarray_csv;
use csv::{ReaderBuilder};
use ndarray_csv::{Array2Reader};
use std::error::Error;
use std::fs::File;

pub type VecC64 = ndarray::Array1<c64>;
pub type MatrixC64 = ndarray::Array2<c64>;

In [8]:
pub fn read_c64_array(csv_file: String, array_size: usize) -> Result<MatrixC64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new().has_headers(false).from_reader(file);
    let array_read: MatrixC64 = reader.deserialize_array2((array_size, array_size))?;
    Ok(array_read)
}

pub fn read_f64_vector(csv_file: String) -> Result<VecF64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new()
        .from_reader(file);

    let v : Vec<f64> = reader.headers()?
        .into_iter()
        .map(|s| s.parse::<f64>().unwrap()) // to f64
        .collect(); // create a vec      
    
    let arr = Array::from(v);
    Ok(arr)
}

let signal: VecF64 = array![1.65531753158569E-06,1.60265876579284E-06,1.54999999999999E-06,1.49734123420715E-06,1.4446824684143E-06];
let idler: VecF64 = array![1.67188028032476E-06,1.61823076531308E-06,1.56458125030141E-06,1.51093173528974E-06,1.45728222027807E-06];
let jsa: MatrixC64 = read_c64_array("jsa_data5.csv".to_string(),5).unwrap();
let dt = 1.0e+06_f64;
let norm = find_two_source_hom(signal.clone(), idler.clone(), jsa.clone(), dt);
println!("norm values are {:?}", norm);
let times = Array::linspace(-300.0e-15_f64, 300.0e-15_f64, 100);
let mut points_time: Vec<f64> = Vec::new();
let mut points_ss: Vec<f64> = Vec::new();
let mut points_ii: Vec<f64> = Vec::new();

for i in 0..99{
    let rate = find_two_source_hom(signal.clone(), idler.clone(), jsa.clone(), times[i]);
    let rate_ss = 0.5*rate.0/norm.0;
    let rate_ii = 0.5*rate.1/norm.1;
    points_time.push(times[i]*1.0e+15_f64);
    points_ss.push(rate_ss);
    points_ii.push(rate_ii);
}
println!("ss is {:?}", points_ss);
println!("ii is {:?}", points_ii);
let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Huong-Ou-Mandel", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(-350_f64..350_f64, 0_f64..1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("Time Delay (fs)")
        .y_desc("Coincidence Probability")
        .draw()?;
    
    chart.draw_series(LineSeries::new(
        points_time.clone().into_iter().zip(points_ss.into_iter()), &RED,))?;
    
    chart.draw_series(LineSeries::new(
        points_time.clone().into_iter().zip(points_ii.into_iter()), &BLUE,))?;

    Ok(())
});
figure

norm values are (25486534343433128000000000000000000000000000000000000000000000.0, 25257886466679666000000000000000000000000000000000000000000000.0, 25729303262655960000000000000000000000000000000000000000000000.0)
ss is [1.0598339855879906, 1.0666310473676053, 1.071364920288838, 1.074013279610629, 1.0745636378204084, 1.0730134033269203, 1.0693698923087362, 1.0636502937006578, 1.0558815875354592, 1.046100417087207, 1.0343529154826145, 1.0206944876573407, 1.0051895487341762, 0.987911220089985, 0.9689409845587439, 0.9483683023902205, 0.9262901897492376, 0.9028107617001647, 0.8780407417765644, 0.852096940387118, 0.8251017044561801, 0.7971823408397314, 0.7684705161937865, 0.7391016361003834, 0.7092142063737992, 0.6789491795738514, 0.6484492898414997, 0.6178583792419473, 0.5873207188502448, 0.5569803278426386, 0.5269802938631746, 0.49746209791958124, 0.4685649470263774, 0.44042511775816884, 0.41317531380446987, 0.386944040531607, 0.36185499945984345, 0.33802650545712803, 0.31557092933668507

Huong-Ou-Mandel
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Coincidence Probability
 
 
Time Delay (fs)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
-300.0
 
 
 
-200.0
 
 
 
-100.0
 
 
 
0.0
 
 
 
100.0
 
 
 
200.0
 
 
 
300.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="93,31 98,31 102,31 107,31 112,31 116,31 121,31 126,31 130,31 135,31 140,31 145,31 149,31 154,33 159,39 163,46 168,53 173,60 177,68 182,76 187,84 191,93 196,102 201,111 205,121 210,130 215,140 219,149 224,159 229,169 233,178 238,187 243,196 247,205 252,214 257,222 261,230 266,237 271,244 275,251 280,257 285,263 289,268 294,272 299,276 303,279 308,282 313,284 317,285 322,286 327,286 332,285 336,284 341,282 346,279 350,276 355,272 360,268 364,263 369,257 374,251 378,244 383,237 388,230 392,222 397,214 402,205 406,196 411,187 416,178 420,169 425,159 430,149 434,140 439,130 444,121 448,111 453,102 458,93 462,84 467,76 472,68 476,60 481,53 486,46 490,39 495,33 500,31 505,31 509,31 514,31 519,31 523,31 528,31 533,31 537,31 542,31 547,31 551,31 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="93,31 98,31 102,31 107,31 112,31 116,31 121,31 126,31 130,31 135,31 140,31 145,31 149,31 154,31 159,37 163,43 168,50 173,57 177,65 182,74 187,82 191,91 196,100 201,109 205,119 210,128 215,138 219,148 224,157 229,167 233,177 238,186 243,195 247,204 252,213 257,221 261,229 266,236 271,244 275,250 280,256 285,262 289,267 294,271 299,275 303,279 308,281 313,283 317,285 322,285 327,285 332,285 336,283 341,281 346,279 350,275 355,271 360,267 364,262 369,256 374,250 378,244 383,236 388,229 392,221 397,213 402,204 406,195 411,186 416,177 420,167 425,157 430,148 434,138 439,128 444,119 448,109 453,100 458,91 462,82 467,74 472,65 476,57 481,50 486,43 490,37 495,31 500,31 505,31 509,31 514,31 519,31 523,31 528,31 533,31 537,31 542,31 547,31 551,31 "/>